# Builtin Complex Refractive Indices of Conductor Materials

We have tabulated complex refractive indices $\vec{n} = n + j\kappa$ for a few
conductor materials in the `data/spd` directory, with $n$ is the refractive
index and $\kappa$ is the absorption coefficient or extinction coefficient.

You can load $n$ and $\kappa$ using `load_builtin_refractive_index`:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import liar
from liar.tools.spd import load_builtin_refractive_index

n, k = load_builtin_refractive_index("Au")
ww = np.array(n.wavelengths)
nn = np.array(n.values)
kk = np.array(k.values)

plt.plot(ww, nn, label="n")
plt.plot(ww, kk, label="k")
plt.xlabel("Wavelength")
plt.ylabel("Value")
plt.legend()
plt.show()

From this, we can use the Fresnel equations to compute the reflectance at
normal indicence:

$R_0 = \frac{n^2 + \kappa^2 - 2n + 1}{n^2 + \kappa^2 + 2n + 1}$

This formula is implemented in `R0Conductor`:

In [ ]:
def R0(n, k):
    return (n * n + k * k - 2 * n + 1) / (n * n + k * k + 2 * n + 1)


r0 = R0(nn, kk)

r = liar.spectra.R0Conductor(n, k)
rr = [r(w) for w in ww]

plt.plot(ww, r0, label="r0")
plt.plot(ww, rr, label="r")
plt.xlabel("Wavelength")
plt.ylabel("Value")
plt.legend()
plt.show()

If we resample `n`, `k` and `r` on finer wavelength resolution (say the
wavelengths $\lambda$ = `ws` of the standard observer), we get a bit of a different result,
because it's `n` and `k` that are linearly interpolated, not `r`!

It's also has a smaller bandwith.

In [ ]:
observer = liar.Observer.standard()
ws = np.array(observer.wavelengths)

ns, ks = n.resample(ws), k.resample(ws)

nns = np.array(ns.values)
kks = np.array(ks.values)
r0s = R0(nns, kks)

rs = liar.spectra.R0Conductor(ns, ks)
rrs = [rs(w) for w in ws]

plt.plot(ww, rr, label="r")
plt.plot(ws, r0s, label="r0s")
plt.plot(ws, rrs, label="rs")
plt.xlabel("Wavelength")
plt.ylabel("Value")
plt.legend()
plt.show()

## Tristimulus values

We can also compute some tristimulus values. Here's the sensitivities
$x$, $y$ and $z$ of the standard observer:

In [ ]:
xs, ys, zs = [np.array(ss) for ss in zip(*observer.sensitivities)]
plt.plot(ws, xs, label="x", color="red")
plt.plot(ws, ys, label="y", color="green")
plt.plot(ws, zs, label="z", color="blue")
plt.xlabel("Wavelength")
plt.ylabel("Sensitivity")
plt.legend()
plt.show()

If we multiply these with the spectrum of our reflectance $r$, we get:

In [ ]:
rxs, rys, rzs = rrs * np.array([xs, ys, zs])

plt.plot(ws, rxs, label="r*x", color="red")
plt.plot(ws, rys, label="r*y", color="green")
plt.plot(ws, rzs, label="r*z", color="blue")
plt.xlabel("Wavelength")
plt.ylabel("Value")
plt.legend()
plt.show()

If we integrate these curves, we get the tristimulus values $R_x, R_y, R_z$:

We use the trapezoidal rule, which corresponds to summing the triangles under
each sample point, with triangle bases $\Delta \lambda_k$ between the previous and
next sample points (except for the first and last one that only use half of
the base), so that we get:

$R_x = \int_{\lambda_{\text{min}}}^{\lambda_{\text{max}}}{rx} = \sum_{k=0}^{n-1} r_k x_k \frac{\Delta \lambda_k}{2}$

with $\Delta \lambda_k = \begin{cases}
\lambda_{k+1} - \lambda_{k-1} \\
\lambda_1-\lambda_0 & \text{if } k=0\\
\lambda_{n-1}-\lambda_{n-2} & \text{if } k=n-1\\
\end{cases}$

In [ ]:
dws = np.concatenate([[ws[1] - ws[0]], ws[2:] - ws[:-2], [ws[-1] - ws[-2]]])

rx = np.sum(rxs * dws / 2)
ry = np.sum(rys * dws / 2)
rz = np.sum(rzs * dws / 2)
print(rx, ry, rz)

This correlates with the built-in means to evaluate the tristimulus values.
The last one is a bit different, as it is based on a linear interpolation of `rr`
instead of `nn` and `kk`.

In [ ]:
print(f"{r.tristimulus=}")
print(f"{observer.tristimulus(rrs)=}")
print(f"{observer.tristimulus(ww, rr)=}")

Calculating the tristimulus values of the reflectance directly based on the
tristimulus values of $n$ and $\kappa$ is not as accurate, but still a good
approximation:

In [ ]:
print(f"{n.tristimulus=}")
print(f"{k.tristimulus=}")
print([R0(n_, k_) for n_, k_ in zip(n.tristimulus, k.tristimulus)])